In [4]:
import pandas as pd 
import os 
from pathlib import Path 


def get_directories(path):
    for subdir, dirs, files in os.walk('./'):
        print(subdir)

# get_directories("./")

cwd = "/home/jupyter/jupyter_default_dir"


# # Path
csv_data = pd.read_csv(f'{cwd}/data/Salaries.csv')
csv_compensation = pd.read_csv(f'{cwd}/data/compensation.csv')
pd.set_option('display.max_columns', 2)


csv_data[["EmployeeName", "Agency"]].head(20)




FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          EmployeeName         Agency
0       NATHANIEL FORD  San Francisco
1         GARY JIMENEZ  San Francisco
2       ALBERT PARDINI  San Francisco
3    CHRISTOPHER CHONG  San Francisco
4      PATRICK GARDNER  San Francisco
5       DAVID SULLIVAN  San Francisco
6            ALSON LEE  San Francisco
7        DAVID KUSHNER  San Francisco
8       MICHAEL MORRIS  San Francisco
9   JOANNE HAYES-WHITE  San Francisco
10       ARTHUR KENNEY  San Francisco
11    PATRICIA JACKSON  San Francisco
12   EDWARD HARRINGTON  San Francisco
13         JOHN MARTIN  San Francisco
14      DAVID FRANKLIN  San Francisco
15     RICHARD CORRIEA  San Francisco
16            AMY HART  San Francisco
17      SEBASTIAN WONG  San Francisco
18          MARTY ROSS  San Francisco
19       ELLEN MOFFATT  San Francisco

In [5]:
type(csv_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

In [15]:
import sys
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.transforms import *
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.types import *
from pyspark.sql import Row
print("##########################Salary##################")
csv_data.info()
print("##########################Copensation##################")
csv_compensation.info()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##########################Salary##################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148654 entries, 0 to 148653
Data columns (total 13 columns):
Id                  148654 non-null int64
EmployeeName        148654 non-null object
JobTitle            148654 non-null object
BasePay             148049 non-null object
OvertimePay         148654 non-null object
OtherPay            148654 non-null object
Benefits            148654 non-null object
TotalPay            148654 non-null float64
TotalPayBenefits    148654 non-null float64
Year                148654 non-null int64
Notes               0 non-null float64
Agency              148654 non-null object
Status              38119 non-null object
dtypes: float64(3), int64(2), object(8)
memory usage: 14.7+ MB
##########################Copensation##################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291825 entries, 0 to 291824
Data columns (total 22 columns):
Year Type                  291825 non-null object
Year

In [7]:
csv_compensation[["Year", "Job"]].head(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Year                             Job
0   2016       Attorney (Civil/Criminal)
1   2013     Senior Management Assistant
2   2015              Eligibility Worker
3   2015                     Dental Aide
4   2014               Recreation Leader
5   2013  Stdntdsgntrain1, Arch/Eng/Plng
6   2015               Library Assistant
7   2014         Real Property Appraiser
8   2015  Lieutenant (Police Department)
9   2014                  Deputy Sheriff
10  2015      Publ Svc Aide-Asst To Prof
11  2013                Street Inspector
12  2015         Park Section Supervisor
13  2013                Transit Operator
14  2016               Personnel Analyst
15  2014         Media/Security Sys Spec
16  2015                Transit Operator
17  2014     Station Agent, Muni Railway
18  2014       Attorney (Civil/Criminal)
19  2015         Water Service Inspector

# DynamicFrame

A DynamicFrame is similar to a DataFrame, except that each record is self-describing, **so no schema is required** initially. Instead, AWS Glue **computes a schema on-the-fly** when required. We convert the df_orders DataFrame into a DynamicFrame.

In [10]:
# Create spark frames
csv_data = csv_data.astype({"Benefits": str}) # Needed to be converted since there was an error with converting types
sparkDF = spark.createDataFrame(csv_data[['Id', 'JobTitle', 'TotalPay', 'Year', 'Benefits']])
sparkDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Id: long (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- TotalPay: double (nullable = true)
 |-- Year: long (nullable = true)
 |-- Benefits: string (nullable = true)

In [13]:
spark_comp = spark.createDataFrame(csv_compensation[["Year", "Job","Total Salary" ]])
spark_comp = spark_comp.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Year: long (nullable = true)
 |-- Job: string (nullable = true)
 |-- Total Salary: double (nullable = true)

In [17]:
glueContext = GlueContext(SparkContext.getOrCreate()) # Generate a glue context from the spark context

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
dyf = DynamicFrame.fromDF(sparkDF, glueContext, "dyf")
dyf.toDF().show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+---------+----+--------+
| Id|            JobTitle| TotalPay|Year|Benefits|
+---+--------------------+---------+----+--------+
|  1|GENERAL MANAGER-M...|567595.43|2011|     nan|
|  2|CAPTAIN III (POLI...|538909.28|2011|     nan|
|  3|CAPTAIN III (POLI...|335279.91|2011|     nan|
|  4|WIRE ROPE CABLE M...|332343.61|2011|     nan|
|  5|DEPUTY CHIEF OF D...|326373.19|2011|     nan|
|  6|ASSISTANT DEPUTY ...|316285.74|2011|     nan|
|  7|BATTALION CHIEF, ...|315981.05|2011|     nan|
|  8|DEPUTY DIRECTOR O...|307899.46|2011|     nan|
|  9|BATTALION CHIEF, ...|303427.55|2011|     nan|
| 10|CHIEF OF DEPARTME...|302377.73|2011|     nan|
+---+--------------------+---------+----+--------+
only showing top 10 rows

# ApplyMapping

The columns in our data might be in different formats, and you may want to change their respective names. ApplyMapping is the best option for changing the names and formatting all the columns collectively

In [19]:
dyf_applymapping = ApplyMapping.apply(frame=dyf , mappings = [
    ("Id", "Long", "id", "String"),
    ("JobTitle", "String", "job_title", "String"),
    ("TotalPay", "Double", "total_pay", "Double"),
    ("Year", "Long", "year", "Long"), 
    ("Benefits", "String", "benefits", "string")
])


                                      
# dyf_applymapping.printSchema()
dyf_applymapping.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------------------+---------+----+--------+
| id|           job_title|total_pay|year|benefits|
+---+--------------------+---------+----+--------+
|  1|GENERAL MANAGER-M...|567595.43|2011|     nan|
|  2|CAPTAIN III (POLI...|538909.28|2011|     nan|
|  3|CAPTAIN III (POLI...|335279.91|2011|     nan|
|  4|WIRE ROPE CABLE M...|332343.61|2011|     nan|
|  5|DEPUTY CHIEF OF D...|326373.19|2011|     nan|
|  6|ASSISTANT DEPUTY ...|316285.74|2011|     nan|
|  7|BATTALION CHIEF, ...|315981.05|2011|     nan|
|  8|DEPUTY DIRECTOR O...|307899.46|2011|     nan|
|  9|BATTALION CHIEF, ...|303427.55|2011|     nan|
| 10|CHIEF OF DEPARTME...|302377.73|2011|     nan|
| 11|ASSISTANT CHIEF O...|299494.17|2011|     nan|
| 12|CAPTAIN III (POLI...|297608.92|2011|     nan|
| 13|EXECUTIVE CONTRAC...|294580.02|2011|     nan|
| 14|   DEPARTMENT HEAD V|292671.62|2011|     nan|
| 15|BATTALION CHIEF, ...|286347.05|2011|     nan|
| 16|COMMANDER III, (P...|286213.86|2011|     nan|
| 17|   DEPARTMENT HEAD V|28472

In [121]:
non_nan = dyf_applymapping.toDF()
non_nan = non_nan[ non_nan["benefits"] != 'nan']
non_nan.show()

An error was encountered:
Invalid status code '404' from http://localhost:8998/sessions/2 with error payload: {"msg":"Session '2' not found."}


# Map

Map allows us to apply a transformation to each record of a Dynamic Frame.


In [26]:
def convert_nan(row):
    if row["benefits"] == 'nan':
        row["is_benefits"] = False
    else:
        row["is_benefits"] = True
    return row

mapped_dyf = Map.apply(frame = dyf_applymapping, f= convert_nan)

mapped_dyf.toDF().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+--------------------+---------+----+---+
|is_benefits|benefits|           job_title|total_pay|year| id|
+-----------+--------+--------------------+---------+----+---+
|      false|     nan|GENERAL MANAGER-M...|567595.43|2011|  1|
|      false|     nan|CAPTAIN III (POLI...|538909.28|2011|  2|
|      false|     nan|CAPTAIN III (POLI...|335279.91|2011|  3|
|      false|     nan|WIRE ROPE CABLE M...|332343.61|2011|  4|
|      false|     nan|DEPUTY CHIEF OF D...|326373.19|2011|  5|
|      false|     nan|ASSISTANT DEPUTY ...|316285.74|2011|  6|
|      false|     nan|BATTALION CHIEF, ...|315981.05|2011|  7|
|      false|     nan|DEPUTY DIRECTOR O...|307899.46|2011|  8|
|      false|     nan|BATTALION CHIEF, ...|303427.55|2011|  9|
|      false|     nan|CHIEF OF DEPARTME...|302377.73|2011| 10|
|      false|     nan|ASSISTANT CHIEF O...|299494.17|2011| 11|
|      false|     nan|CAPTAIN III (POLI...|297608.92|2011| 12|
|      false|     nan|EXECUTIVE CONTRAC...|294580.02|20

In [33]:
df = dyf_applymapping.toDF()
maximum = df["year"]

type(maximum)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.column.Column'>